In [18]:
import pandas_datareader.data as web
import pandas as pd
import datetime
from decimal import Decimal
from dateutil import relativedelta
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, resample_apply
import importlib
import plotly.express as px

#Change me!
start = datetime.datetime.today() + relativedelta.relativedelta(years=-10)
#Mで入力する
split_unit = "12M"
modulename = "BacktestingStrategies.Strategy_MacdCross"
classname = "btst.MACDCross"
end = datetime.date.today()

data = web.DataReader('2558.T', 'yahoo', start, end)
data = data.astype("double")

initial = 1000_000
fname = f"html/plot-"

In [19]:
#日付データを検証期間単位ごとにSplit
#https://kakakakakku.hatenablog.com/entry/2021/05/24/002542
grouped_data = data.groupby(pd.Grouper(freq = split_unit))
dfs = [group for _, group in grouped_data]

In [20]:
#検証期間単位ごとにバックテスト＆結果を蓄積
#親ディレクトリ経由で呼び出し   # https://qiita.com/yokohama4580/items/466a483ae022d264c8ee
import os
import sys
import re
sys.path.append(os.pardir)

btst = importlib.import_module(modulename)
# from BacktestingStrategies import Strategy_RsiOscillator as btst
importlib.reload(btst)
import warnings
warnings.simplefilter('ignore')

warmup = int(re.sub(r"\D", "", split_unit)) / 2 * 20

returns = {}
bts = {}
i = 0

for monthly_data in dfs:
    if i != 0:
        start = dfs[i - 1].tail(int(warmup))
        end = data[data.index == monthly_data.tail(1).index.tolist()[0]]
        monthly_data = pd.concat([start, end])

    
    # バックテストを設定
    bt = Backtest(
        monthly_data,
        eval(classname), # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True #自動でポジションをクローズ(オープン)
    )

    output = bt.run() # バックテスト実行
    returns[bt] = output
    # bts[str(i)] = bt

    i = i + 1

In [21]:
#ベスト１、ワースト１、ワースト３を抽出
returns_without0trades = {k: v for k, v in returns.items() if v["# Trades"] != 0}
returns_sorted = sorted(returns_without0trades.items(), key=lambda x: x[1]["Return [%]"])

min_return_3 = returns_sorted[:3]
top1_return = returns_sorted[-1:]

max_return = str((top1_return[0][1])["Return [%]"])
max_period_start = str((top1_return[0][1])["Start"])
max_period_end = str((top1_return[0][1])["End"])

min_return = str((min_return_3[0][1])["Return [%]"])
min_period_start = str((min_return_3[0][1])["Start"])
min_period_end = str((min_return_3[0][1])["End"])

In [22]:
#まとめを表示
import statistics
df = pd.DataFrame(returns.values())

lines = "１期間単位: " + split_unit + "\n"
lines += "テスト期間: " + str(len(returns)) + "\n"
res = round(statistics.mean(df["Return [%]"]), 4)

lines += "平均Return [%]: " + str(res) + "\n"
lines += "最大Return [%]: " + str(max_return) + "\n"
lines += "最大Return期間: " + str(max_period_start) + " - " + str(max_period_end) + "\n"
lines += "最低Return [%]: " + str(min_return) + "\n"
lines += "最低Return期間: " + str(min_period_start) + " - " + str(min_period_end) 
print(lines)

１期間単位: 12M
テスト期間: 4
平均Return [%]: 0.7595
最大Return [%]: 3.2447661249999893
最大Return期間: 2021-08-05 00:00:00 - 2022-10-18 00:00:00
最低Return [%]: -0.20669399999999444
最低Return期間: 2020-08-04 00:00:00 - 2022-01-31 00:00:00


In [23]:
#ワースト３を表示
i = 1
for min_data in min_return_3:
    print("----- WORST " + str(i) + "-----")
    display(min_data[1])
    display(min_data[1]._trades)
    min_data[0].plot(filename = fname + str(min_data[1]._strategy) + "_min" + str(i) + ".html")

    i = i + 1

----- WORST 1-----


Start                     2020-08-04 00:00:00
End                       2022-01-31 00:00:00
Duration                    545 days 00:00:00
Exposure Time [%]                    13.22314
Equity Final [$]                    997933.06
Equity Peak [$]                   1023618.805
Return [%]                          -0.206694
Buy & Hold Return [%]               47.347347
Return (Ann.) [%]                   -0.429989
Volatility (Ann.) [%]                 5.43647
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                    -2.78187
Avg. Drawdown [%]                   -1.539995
Max. Drawdown Duration      475 days 00:00:00
Avg. Drawdown Duration      239 days 00:00:00
# Trades                                    2
Win Rate [%]                             50.0
Best Trade [%]                       0.945407
Worst Trade [%]                     -1.143545
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,95,43,50,10431.381,10530.0,9368.805,0.009454,2020-10-08,2020-10-19,11 days
1,93,65,72,10752.965,10630.0,-11435.745,-0.011435,2020-11-10,2020-11-19,9 days


----- WORST 2-----


Start                     2021-08-05 00:00:00
End                       2022-10-18 00:00:00
Duration                    439 days 00:00:00
Exposure Time [%]                    8.264463
Equity Final [$]                1032447.66125
Equity Peak [$]                   1044790.095
Return [%]                           3.244766
Buy & Hold Return [%]               14.574315
Return (Ann.) [%]                    6.876497
Volatility (Ann.) [%]                4.631858
Sharpe Ratio                         1.484609
Sortino Ratio                        3.890672
Calmar Ratio                         5.820971
Max. Drawdown [%]                   -1.181331
Avg. Drawdown [%]                   -0.763559
Max. Drawdown Duration      361 days 00:00:00
Avg. Drawdown Duration      122 days 00:00:00
# Trades                                    1
Win Rate [%]                            100.0
Best Trade [%]                       3.931552
Worst Trade [%]                      3.931552
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,70,45,54,14240.1415,14800.0,39190.095,0.039316,2021-10-12,2021-10-25,13 days


In [24]:
#ベストを表示
print("----- BEST -----")
display(top1_return[0][1])
display(top1_return[0][1]._trades)
top1_return[0][0].plot(filename = fname + str(top1_return[0][1]._strategy) + "_best" + str(i) + ".html")

----- BEST -----


Start                     2021-08-05 00:00:00
End                       2022-10-18 00:00:00
Duration                    439 days 00:00:00
Exposure Time [%]                    8.264463
Equity Final [$]                1032447.66125
Equity Peak [$]                   1044790.095
Return [%]                           3.244766
Buy & Hold Return [%]               14.574315
Return (Ann.) [%]                    6.876497
Volatility (Ann.) [%]                4.631858
Sharpe Ratio                         1.484609
Sortino Ratio                        3.890672
Calmar Ratio                         5.820971
Max. Drawdown [%]                   -1.181331
Avg. Drawdown [%]                   -0.763559
Max. Drawdown Duration      361 days 00:00:00
Avg. Drawdown Duration      122 days 00:00:00
# Trades                                    1
Win Rate [%]                            100.0
Best Trade [%]                       3.931552
Worst Trade [%]                      3.931552
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,70,45,54,14240.1415,14800.0,39190.095,0.039316,2021-10-12,2021-10-25,13 days


Row(id='13902', ...)

In [25]:
#散布図を表示
fig = px.box(df["Return [%]"], points = "all")
fig.update_layout(
    xaxis_title = "Strategy",
    yaxis_title = "Return [%]"
)
fig.show()